In [2]:
import librosa
import numpy as np
import os

In [3]:
filter_path = '../../filters/huang/second_most_common.txt'

In [4]:
def load_data(filter_path):
    file_path = None
    with open(filter_path, 'r') as f:
        file_path = f.readlines()
        file_path = map(str.strip, file_path)
        file_path = list(file_path)
    
    audio_data = []
    
    for file in file_path:
        audio, _ = librosa.load(file, sr=None)
        audio_data.append(audio)
    
    audio_data = np.array(audio_data)
    return audio_data
    
audio_data = load_data(filter_path)
audio_data.shape

(1998, 177152)

In [5]:
audio_data.shape
test_audio_data = audio_data.reshape(-1, 1)
test_audio_data.shape

(353949696, 1)

In [29]:
from itertools import islice

def to_window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield np.array(result)
    for elem in it:
        result = result[1:] + (elem,)
        yield np.array(result)

def get_window_batch(seq, window_size, batch_size):
    it = 0
    d = []
    
    gen = to_window(seq, window_size + 1)
    while True:
        while it < batch_size:
            try:
                data = next(gen)
            except:
                gen = to_window(seq, window_size)
                data = next(gen)
            d.append(data)
            it += 1
        it = 0
        d = np.array(d)
        X = d[:, : window_size]
        y = d[:, -1]
        yield (X, y)
        d = []

In [40]:
test_audio_data.shape

(353949696, 1)

In [38]:
gen = get_window_batch(test_audio_data[1000000:1020000], window_size=2, batch_size=10)